In [1]:
import json
import h5py
import numpy as np
import csv

test_path = "/home/maelic/Documents/PhD/Datasets/VisualGenome/anno_sg2im_format/test.h5"
vocab = json.load(open("/home/maelic/Documents/PhD/Datasets/VisualGenome/anno_sg2im_format/vocab.json", 'r'))
h5_file = h5py.File(test_path, 'a')

cat_path = "/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/category_classification/vg150_classification/categories_gpt3.5_vg150_2.csv"
categories = csv.reader(open(cat_path, 'r'))

#get average number of rel per image
print(np.average(h5_file['relationships_per_image']))
print(h5_file.keys())

4.164070180745671
<KeysViewHDF5 ['attributes_per_object', 'image_ids', 'image_paths', 'object_attributes', 'object_boxes', 'object_ids', 'object_names', 'objects_per_image', 'relationship_ids', 'relationship_objects', 'relationship_predicates', 'relationship_subjects', 'relationships_per_image']>


In [2]:
print(categories)
cat_dict = {}
for row in categories:
    cat_dict[row[0]] = row[1]

In [3]:
from tqdm import tqdm
# for each relationship, get the category of the subject and object
# get object ids
for img_idx in tqdm(range(len(h5_file['relationship_ids']))):
    obj_ids = h5_file['object_ids'][img_idx]
    # get object names
    obj_names = h5_file['object_names'][img_idx]
    # get attributes
    obj_attributes = h5_file['object_attributes'][img_idx]
    # get relationship ids
    rel_ids = h5_file['relationship_ids'][img_idx]
    # get relationship objects
    rel_objs = h5_file['relationship_objects'][img_idx]
    # get relationship predicates
    rel_preds = h5_file['relationship_predicates'][img_idx]
    # get relationship subjects
    rel_subs = h5_file['relationship_subjects'][img_idx]

    for i, rel in enumerate(rel_ids):
        if rel == -1:
            break
        sub = rel_subs[i]
        obj = rel_objs[i]
        pred = rel_preds[i]
        sub_name = vocab['object_idx_to_name'][obj_names[sub]]
        obj_name = vocab['object_idx_to_name'][obj_names[obj]]
        pred_name = vocab['pred_idx_to_name'][pred]

        full_rel = sub_name + " " + pred_name + " " + obj_name
        cat = cat_dict[full_rel]
        if cat == "attribute":
            # remove relationship
            rel_ids[i] = -1
            rel_objs[i] = -1
            rel_preds[i] = -1
            rel_subs[i] = -1
    
    # reorganize the arrays such as all -1 values are moved at the end
    # get number of -1 values
    nb_rel = len(rel_ids) - np.count_nonzero(rel_ids == -1)
    # get indexes of non -1 values
    idx2 = np.where(rel_ids != -1)
    # new arrrays with those values
    rel_ids = rel_ids[idx2]
    rel_objs = rel_objs[idx2]
    rel_preds = rel_preds[idx2]
    rel_subs = rel_subs[idx2]
    # pad the arrays with -1 values until 30
    rel_ids = np.pad(rel_ids, (0, 30 - nb_rel), 'constant', constant_values=-1)
    rel_objs = np.pad(rel_objs, (0, 30 - nb_rel), 'constant', constant_values=-1)
    rel_preds = np.pad(rel_preds, (0, 30 - nb_rel), 'constant', constant_values=-1)
    rel_subs = np.pad(rel_subs, (0, 30 - nb_rel), 'constant', constant_values=-1)

    h5_file['relationships_per_image'][img_idx] = nb_rel

print(np.average(h5_file['relationships_per_image']))

out_path = "/home/maelic/Documents/PhD/ModelZoo/SGG2IM/sg2im/datasets/experiments/vg150_no_attribute.h5"

# copy and write down to new file
with h5py.File(out_path, 'w') as f:
    for k, v in h5_file.items():
        f.create_dataset(k, data=v)

100%|██████████| 26446/26446 [00:16<00:00, 1622.36it/s]

6.237351584360584
